In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# with open('/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt') as file:
    
#     lines=file.readlines()
# lines

In [3]:
# train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
# train_df = train_df.drop(columns=['Id'])




In [4]:
train_df.shape

NameError: name 'train_df' is not defined

In [ ]:
missing_train = train_df.isna().sum().sort_values(ascending=False)
missing_train = missing_train[missing_train > 0]
print("\nMissing Values in Train Data:")
print(missing_train)


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train_df['SalePrice'], kde=True, bins=30, color='blue')
plt.title('Distribution of SalePrice', fontsize=14)
plt.xlabel('SalePrice', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

In [ ]:
train_df['LogSalePrice'] = np.log1p(train_df['SalePrice'])
plt.figure(figsize=(10, 6))
sns.histplot(train_df['LogSalePrice'], kde=True, bins=30, color='green')
plt.title('Log-Transformed SalePrice', fontsize=14)
plt.xlabel('LogSalePrice', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

In [ ]:
train_df_encoded = pd.get_dummies(train_df)
corr_matrix = train_df_encoded.corr()
top_corr_features = corr_matrix['SalePrice'].sort_values(ascending=False)

In [ ]:
print("\nTop Features Correlated with SalePrice:")
print(top_corr_features.head(10))

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix[['SalePrice']].sort_values(by='SalePrice', ascending=False), 
            annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation of Features with SalePrice', fontsize=14)
plt.show()

In [ ]:
key_features = ['GrLivArea', 'OverallQual', 'TotalBsmtSF']
for feature in key_features:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=train_df, x=feature, y='SalePrice', alpha=0.8, color='purple')
    plt.title(f'{feature} vs SalePrice', fontsize=14)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('SalePrice', fontsize=12)
    plt.show()

In [ ]:
categorical_features = train_df.select_dtypes(include=['object']).columns
categorical_features

In [ ]:
# missing_train_cat = missing_train[ train_df.select_dtypes(include=['object']).columns]
# missing_train_cat

In [ ]:
train_df[categorical_features]

In [ ]:
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train_df.select_dtypes(include=['object']).columns
num_imputer = SimpleImputer(strategy='constant', fill_value=0)  # Use 0 for numerical features
cat_imputer = SimpleImputer(strategy='constant',fill_value='missing')
train_df[numerical_features] = num_imputer.fit_transform(train_df[numerical_features])
train_df[categorical_features] = cat_imputer.fit_transform(train_df[categorical_features])

In [ ]:
numerical_features=numerical_features.drop(["SalePrice","LogSalePrice"])
numerical_features

In [ ]:
numerical_features

In [ ]:
missing_values = train_df.isna().sum()
missing_values[missing_values>0]




In [ ]:
Q1 = train_df[numerical_features].quantile(0.25)
Q3 = train_df[numerical_features].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = ((train_df[numerical_features] < lower_bound) | (train_df[numerical_features] > upper_bound))

outliers_count = outliers.sum()
outliers_count[outliers_count>0]


In [ ]:
from scipy import stats

# Calculate the Z-scores for numerical features
z_scores = stats.zscore(train_df[numerical_features])

# Identify the outliers (z-scores > 3 or < -3)
outliers_z = (z_scores > 3) | (z_scores < -3)

# Count the number of outliers for each feature
outliers_z_count = outliers_z.sum(axis=0)
print("Number of outliers based on Z-scores:")
print(outliers_z_count)


In [ ]:
# If your target is a classification label (e.g., 'SaleCondition' or similar)
class_distribution = train_df['SalePrice'].value_counts()  # Replace with your target column
print("Class distribution for target variable:")
print(class_distribution)

# Visualize the class distribution
import matplotlib.pyplot as plt

class_distribution.plot(kind='bar')
plt.title('Class Distribution')
plt.xlabel('Classes')
plt.ylabel('Count')
plt.show()


In [ ]:
# Plot histogram for SalePrice
train_df['SalePrice'].hist(bins=50)
plt.title('Distribution of SalePrice')
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()


In [ ]:
skewness = train_df['SalePrice'].skew()
print(f'Skewness of SalePrice: {skewness}')


In [ ]:
skewed_features = train_df[numerical_features].skew().sort_values(ascending=False)
skewed_features = skewed_features[skewed_features > 0.5].index  # Adjust threshold if needed
skewed_features_test = skewed_features[skewed_features != 'SalePrice']
train_df[skewed_features] = np.log1p(train_df[skewed_features])
test_df[skewed_features_test] = np.log1p(test_df[skewed_features_test])

In [ ]:
skewness = train_df[numerical_features].skew()
print(f'Skewness of SalePrice: {skewness}')

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
scaler = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', num_imputer), ('scaler', scaler)]), numerical_features),
        ('cat', Pipeline(steps=[('imputer', cat_imputer), ('ohe', ohe)]), categorical_features)
    ]
)

In [ ]:
X = train_df.drop(['SalePrice', 'LogSalePrice'], axis=1,errors='ignore')
y = train_df['LogSalePrice']
X_test = test_df

In [ ]:
# Check the column names in train_df
print(X.columns)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=40)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 4.1: Model Training

# Initialize the models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate each model
results = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    # Train the model
    model.fit(X_train_preprocessed, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val_preprocessed)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    results[model_name] = rmse
    print(f"RMSE for {model_name}: {rmse:.4f}")

# Display results
print("\nModel Comparison (RMSE):")
for model_name, rmse in results.items():
    print(f"{model_name}: {rmse:.4f}")

In [ ]:
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Step 1: Train the model
lgb_model = lgb.LGBMRegressor(n_estimators=10000, learning_rate=0.05 ,max_depth=50, subsample=0.7, colsample_bytree=0.8, random_state=40)

# Assuming X_train_preprocessed and y_train are already defined and X_train_preprocessed is a NumPy array
X_train_preprocessed = pd.DataFrame(X_train_preprocessed)  # Convert to DataFrame
lgb_model.fit(X_train_preprocessed, y_train)

# Step 2: Make predictions on the validation set and calculate RMSE
y_val_pred = lgb_model.predict(X_val_preprocessed)

rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"RMSE for LGBM: {rmse:.4f}")

# Step 3: Save the trained model
joblib.dump(lgb_model, 'lgb_model.pkl')

# Step 4: Load the saved model and make predictions on the test set

# Assuming test_df is your test data
test_file_path = "../input/house-prices-advanced-regression-techniques/test.csv"
test_df = pd.read_csv(test_file_path)
idx = test_df.pop('Id')  # Pop 'Id' before processing the test data

# Ensure the test set has the same columns as the training set
missing_cols = set(X_train_preprocessed.columns) - set(test_df.columns)

# Create a DataFrame with missing columns and default values
missing_data = pd.DataFrame({col: 0 for col in missing_cols}, index=test_df.index)

# Concatenate the missing columns to the test DataFrame
test_df = pd.concat([test_df, missing_data], axis=1)

# Reorder test set columns to match training set
test_df = test_df[X_train_preprocessed.columns]

# Step 5: Load the saved model
lgb_model_loaded = joblib.load('lgb_model.pkl')

# Step 6: Predict on the test set
y_test_pred = lgb_model_loaded.predict(test_df)

# Create the output DataFrame


In [ ]:
output = pd.DataFrame({'Id': idx, 'SalePrice': y_test_pred})

output.to_csv('/kaggle/working/submission.csv', index=False)

print(output.head())
